# Pair Programming ETL II

Tendréis que usar el csv attacks_limpieza_completa que tenéis adjunto abajo.
En la lección de hoy aprendimos como transformar nuestros datos para que estén preparados para almacearlos en una BBDD. En este momento tenemos dos fuentes de datos:
1. El csv con los ataques de tiburones que hemos estado limpiando hasta ahora, el que os hemos adjuntado (attacks_limpieza_completa). Sentiros libres de usar vuestros propios csv en caso de que queráis.
2. El csv con los datos climáticos de los principales paises que tienen ataques de tiburones, el que creamos en el pair programming de ayer.

El objetivo de la sesión de hoy será juntar en un único csv la información de ambas fuentes. Para ello:
- Cargaremos los dos ficheros de datos
- Del dataframe de los ataques nos quedaremos solo con las filas de los países que seleccionamos en la lección de ayer:
    -  USA
    -  Australia
    -  New Zealand
    -  South Africa
    -  Papua New Guinea
- Del dataframe de los datos climáticos seleccionaremos todas las columnas.
- Cuando ya tengamos todos los datos deseados juntaremos los dos csv.
    - Para hacer esta unión tendremos que hacer un groupby en la tabla de clima para sacar una media de las medidas climáticas por país.
    - Antes de hacer el groupby si nos fijamos tenemos dos columnas rh_profile y wind_profile cuya información es una lista de diccionarios. Si intentamos hacer la media de eso no nos dará un valor real. A este problema ya nos enfrentamos en la clase invertida de ETL-2, donde teníais un Bonus para desempaquetar esta información. En caso de que en aquel ejercicio no lo consigierais os dejamos por aquí una posible solución que nos permite separar esa información en distintas columnas. Os dejamos el código documentado. ⚠️ Os recomendamos que vayáis desgranando el código y viendo lo que nos devuelve cada línea de código para entenderlo mejor.

In [62]:
from IPython.core.interactiveshell import InteractiveShell 
InteractiveShell.ast_node_interactivity = "all"

import requests
import pandas as pd
import numpy as np



In [63]:
pd.options.display.max_columns = None

In [64]:
df_attack = pd.read_csv("../datos/datos_limpiezaV.csv", index_col=0)
df_clima = pd.read_csv("../datos/datos_clima.csv",index_col=0)

In [65]:
df_clima.head()

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-4,11,-9999,none,1,0,205,2,usa
1,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 275, 'speed':...",27,-4,11,-9999,rain,1,0,240,2,usa
2,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,265,2,usa
3,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,200,2,usa
4,15,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,none,1,0,195,3,usa


In [66]:
df_attack.head()

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
1,2018.0,brazil,18,Tiger shark,03-Jun-2018,Jun,y,M,tiger_shark,18.0,18.0,M
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
4,2018.0,england,21,Invalid incident,13-May-2018,May,n,M,unspecified,21.0,21.0,M


In [67]:
attaques = df_attack[(df_attack["country"] == "usa") | (df_attack["country"] == "australia") | (df_attack["country"] == "papua new guinea") | (df_attack["country"] == "new zealand")| (df_attack["country"] == "south africa")]
attaques.head()
                     

,year,country,age,species,date,month,fatal,sex,main_species,age_numbers,age_numbers_mean,sex_moda
0,2018.0,usa,57,White shark,25-Jun-2018,Jun,n,F,white_shark,57.0,57.0,F
2,2018.0,usa,15,"Bull shark, 6'",26-May-2018,May,n,M,bull_shark,15.0,15.0,M
3,2018.0,australia,32,Grey reef shark,24-May-2018,May,n,M,grey_shark,32.0,32.0,M
6,2018.0,australia,60,3 m shark,25-Apr-2018,Apr,n,M,unspecified,60.0,60.0,M
8,2018.0,south africa,33,"White shark, 2.5 m",22-Apr-2018,Apr,n,M,white_shark,33.0,33.0,M


In [68]:
# en Python tenemos la librería `ast` que nos permite castear un string que dentro tiene diccionarios, o listas o tuplas a su tipo correspondiente. 
# En nuestro caso, lo que conseguiremos es no tener strings sino listas en la columna. Esto lo haremos de la siguiente forma: 

import ast

df_clima["wind_profile"]= df_clima["wind_profile"].apply(ast.literal_eval)

# una vez que tengamos la columna cambiada, una fantasía de Pandas es que si hago un apply sobre una columna cuyos valores son diccionarios o 
# listas nos va a genererar una columna con los valores de los diccionarios o listas. Donde cada columna será key del diccionario o cada elemento de la lista. 


wind = df_clima['wind_profile'].apply(pd.Series)



In [69]:

# nos creamos un dataframe nuevo con el resultado de la información de una de las columnas separadas por columnas. 
# Esto nos va a devolver un dataframe donde cada fila será una celda del dataframe anterior. 
df_clima["rh_profile"]= df_clima["rh_profile"].apply(ast.literal_eval)
rh = df_clima['rh_profile'].apply(pd.Series) 

In [71]:
df_clima.head(1)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-4,11,-9999,none,1,0,205,2,usa


In [72]:
rh.head(1)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 11}","{'layer': '800mb', 'rh': 9}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 7}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 9}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 13}","{'layer': '400mb', 'rh': 12}","{'layer': '350mb', 'rh': 7}","{'layer': '300mb', 'rh': 7}","{'layer': '250mb', 'rh': 12}","{'layer': '200mb', 'rh': 16}"


In [73]:
rh.head(1)

for i in range(len(rh.columns)): 
    #x = rh[i].apply(pd.Series)
    #print (x)

    # aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
    nombre = "rh_" + str(rh[i].apply(pd.Series)["layer"][0]) 
    print(nombre)
    # hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
    #valores = list(rh[i].apply(pd.Series)["rh"] )

    # usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
    #df_clima.insert(i, nombre, valores)
    


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,"{'layer': '950mb', 'rh': 12}","{'layer': '900mb', 'rh': 12}","{'layer': '850mb', 'rh': 11}","{'layer': '800mb', 'rh': 9}","{'layer': '750mb', 'rh': 8}","{'layer': '700mb', 'rh': 7}","{'layer': '650mb', 'rh': 4}","{'layer': '600mb', 'rh': 6}","{'layer': '550mb', 'rh': 9}","{'layer': '500mb', 'rh': 14}","{'layer': '450mb', 'rh': 13}","{'layer': '400mb', 'rh': 12}","{'layer': '350mb', 'rh': 7}","{'layer': '300mb', 'rh': 7}","{'layer': '250mb', 'rh': 12}","{'layer': '200mb', 'rh': 16}"


rh_0    950mb
0    950mb
0    950mb
0    950mb
0    950mb
Name: layer, dtype: object
rh_0    900mb
0    900mb
0    900mb
0    900mb
0    900mb
Name: layer, dtype: object
rh_0    850mb
0    850mb
0    850mb
0    850mb
0    850mb
Name: layer, dtype: object
rh_0    800mb
0    800mb
0    800mb
0    800mb
0    800mb
Name: layer, dtype: object
rh_0    750mb
0    750mb
0    750mb
0    750mb
0    750mb
Name: layer, dtype: object
rh_0    700mb
0    700mb
0    700mb
0    700mb
0    700mb
Name: layer, dtype: object
rh_0    650mb
0    650mb
0    650mb
0    650mb
0    650mb
Name: layer, dtype: object
rh_0    600mb
0    600mb
0    600mb
0    600mb
0    600mb
Name: layer, dtype: object
rh_0    550mb
0    550mb
0    550mb
0    550mb
0    550mb
Name: layer, dtype: object
rh_0    500mb
0    500mb
0    500mb
0    500mb
0    500mb
Name: layer, dtype: object
rh_0    450mb
0    450mb
0    450mb
0    450mb
0    450mb
Name: layer, dtype: object
rh_0    400mb
0    400mb
0    400mb
0    400mb
0    400mb
Name: l

In [47]:
df_clima.head()

,"rh_0 950mb\n0 950mb\n0 950mb\n0 950mb\n0 950mb\nName: layer, dtype: object","rh_0 900mb\n0 900mb\n0 900mb\n0 900mb\n0 900mb\nName: layer, dtype: object","rh_0 850mb\n0 850mb\n0 850mb\n0 850mb\n0 850mb\nName: layer, dtype: object","rh_0 800mb\n0 800mb\n0 800mb\n0 800mb\n0 800mb\nName: layer, dtype: object","rh_0 750mb\n0 750mb\n0 750mb\n0 750mb\n0 750mb\nName: layer, dtype: object","rh_0 700mb\n0 700mb\n0 700mb\n0 700mb\n0 700mb\nName: layer, dtype: object","rh_0 650mb\n0 650mb\n0 650mb\n0 650mb\n0 650mb\nName: layer, dtype: object","rh_0 600mb\n0 600mb\n0 600mb\n0 600mb\n0 600mb\nName: layer, dtype: object","rh_0 550mb\n0 550mb\n0 550mb\n0 550mb\n0 550mb\nName: layer, dtype: object","rh_0 500mb\n0 500mb\n0 500mb\n0 500mb\n0 500mb\nName: layer, dtype: object","rh_0 450mb\n0 450mb\n0 450mb\n0 450mb\n0 450mb\nName: layer, dtype: object","rh_0 400mb\n0 400mb\n0 400mb\n0 400mb\n0 400mb\nName: layer, dtype: object","rh_0 350mb\n0 350mb\n0 350mb\n0 350mb\n0 350mb\nName: layer, dtype: object","rh_0 300mb\n0 300mb\n0 300mb\n0 300mb\n0 300mb\nName: layer, dtype: object","rh_0 250mb\n0 250mb\n0 250mb\n0 250mb\n0 250mb\nName: layer, dtype: object","rh_0 200mb\n0 200mb\n0 200mb\n0 200mb\n0 200mb\nName: layer, dtype: object",timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,12,12,11,9,8,7,4,6,9,14,13,12,7,7,12,16,3,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 180, 'speed':...",27,-4,11,-9999,none,1,0,205,2,usa
1,12,11,11,9,7,7,5,10,9,13,10,7,15,9,11,14,6,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 12}, {'layer': '900m...","[{'layer': '950mb', 'direction': 275, 'speed':...",27,-4,11,-9999,rain,1,0,240,2,usa
2,11,11,11,10,7,6,8,8,11,14,11,7,10,5,8,15,9,9,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,265,2,usa
3,11,10,9,7,7,11,12,12,12,11,12,9,9,10,9,15,12,7,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 11}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,rain,1,0,200,2,usa
4,13,11,10,9,9,12,12,9,8,10,13,8,13,14,15,13,15,5,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 13}, {'layer': '900m...","[{'layer': '950mb', 'direction': 195, 'speed':...",27,-1,10,-9999,none,1,0,195,3,usa
